In [1]:
import pandas as pd
import numpy as np
import pytimetk as pytimetk
import random
import matplotlib.pyplot as plt
import os
import datetime
from datetime import date, timedelta 
from datetime import date, timedelta 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error




ModuleNotFoundError: No module named 'pandas'

In [2]:


# File Path


max_temp_file_path = 'G:/fresh_start/main_thesis/data/raw/Max_temp.xlsx'




In [3]:
#reading workbook
initial_max_temp_df = pd.ExcelFile(max_temp_file_path)


### For maximum temperature

In [ ]:
max_file_column_order =  [ 'Station Index', 'Max T']
initial_max_dict = {}
# Iterate over each sheet and create a DataFrame
for sheet_name in initial_max_temp_df.sheet_names:
    max_df_parsed = initial_max_temp_df.parse(sheet_name)

    initial_max_dict[sheet_name] = max_df_parsed
    



     # Converting a Year , Month and Day Column into Date column
    max_df_parsed['Date'] = pd.to_datetime(max_df_parsed[['Year', 'Month', 'Day']])

    

    # Set "Date" as the index column
    max_df_parsed.set_index('Date', inplace=True)

    max_df_parsed = max_df_parsed[max_file_column_order]
    
    max_df_parsed= max_df_parsed.rename(columns={'Station Index': 'Station'})
    
    initial_max_dict[sheet_name] = max_df_parsed



In [ ]:
all_station_as_it_is_data_frame_1 = []
for station_name, max_temp_df in initial_max_dict.items():
    max_temp_df.loc[:, 'Station'] = station_name
    all_station_as_it_is_data_frame_1.append(max_temp_df)


all_station_as_it_is_data_frame_2 = pd.concat(all_station_as_it_is_data_frame_1, ignore_index=False)

all_station_as_it_is_data_frame_2['Max T'] = all_station_as_it_is_data_frame_2['Max T'].replace('NA', np.nan)

In [ ]:
max_outlier_check_dict = {}

for station_name, df_max_temp in initial_max_dict.items():
    


    # Calculate the IQR for 'Max T' column
    Q1 = df_max_temp['Max T'].quantile(0.25)
    Q3 = df_max_temp['Max T'].quantile(0.75)
    IQR = Q3 - Q1

    # Define lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    

    # Filter out data points outside the bounds
    df_max_temp = df_max_temp[(df_max_temp['Max T'] >= lower_bound) & (df_max_temp['Max T'] <= upper_bound)]
    max_outlier_check_dict[station_name]=df_max_temp



In [ ]:
def generate_dummy_temp_data( station_code):
    # Create an empty list to store the data
    data = []

    # Generate random temperature data for each day within the date range
    current_date = date(1962, 1, 1)
    while current_date <= date(2022, 12, 31):
        temperature = round(random.uniform(1900, 2000), 1)
        data.append([current_date, station_code, temperature])
        current_date += timedelta(days=1)

    # Create a DataFrame from the generated data
    dummy_temp_df = pd.DataFrame(data, columns=['Date', 'Station', 'Max_Temp'])

    # Set the 'Date' column as the index
    #dummy_temp_df.set_index('Date', inplace=True)

    return dummy_temp_df



In [ ]:
max_temp_1960_2022_initial = {}

for station_name, max_temp_df in max_outlier_check_dict.items():
    
    max_temp_df = max_temp_df.reset_index()
    max_temp_df['Date'] = pd.to_datetime(max_temp_df['Date'])
    
    dummy_temp_df = generate_dummy_temp_data(station_name)
    
    dummy_temp_df['Date'] = pd.to_datetime(dummy_temp_df['Date'])
    
    df3 = pd.merge(dummy_temp_df, max_temp_df, on=['Date'], how='left').fillna('NA')
    
    
    max_temp_1960_2022_initial[station_name] = df3
    
    

In [ ]:
for station_name, max_temp_df in max_temp_1960_2022_initial.items():
    max_temp_df = max_temp_df[['Date', 'Max T']]
    max_temp_1960_2022_initial[station_name] = max_temp_df

In [ ]:
# Initializing data frames
all_station_data_initial_df = []

for station_name, max_temp_df in max_temp_1960_2022_initial.items():
    max_temp_df.loc[:, 'Station'] = station_name
    all_station_data_initial_df.append(max_temp_df)

result_df_max_temp = pd.concat(all_station_data_initial_df, ignore_index=True)

result_df_max_temp = result_df_max_temp[['Date', 'Station', 'Max T']]
# Replace 'NA' with NaN
result_df_max_temp['Max T'] = result_df_max_temp['Max T'].replace('NA', np.nan)
    

In [ ]:

result_df_max_temp['Date'] = pd.to_datetime(result_df_max_temp['Date'])

start_date = pd.to_datetime('1990-01-01')
end_date = pd.to_datetime('2022-12-31')

# Filter the DataFrame to include data within the specified date range
filtered_data_df = result_df_max_temp[(result_df_max_temp['Date'] >= start_date) & (result_df_max_temp['Date'] <= end_date)]



In [ ]:
max_temp_dict_by_month1 = {}
unique_months_max_df = filtered_data_df['Date'].dt.month.unique()

for month in unique_months_max_df:
    monthly_max_temp = filtered_data_df[filtered_data_df['Date'].dt.month == month].copy()
    
    monthly_max_temp = monthly_max_temp.reindex()
    
    max_temp_dict_by_month1[month] = monthly_max_temp
    
    
 

In [ ]:

def impute_missing_with_regression(df, target_column, feature_columns):
    # Split data into training and prediction data
    training_data = df.dropna(subset=[target_column] + feature_columns)
    prediction_data = df[df[target_column].isna()]

    # Create and train a regression model
    X_train = training_data[feature_columns]
    y_train = training_data[target_column]
    regression_model = LinearRegression()
    regression_model.fit(X_train, y_train)

    # Predict missing values
    X_pred = prediction_data[feature_columns]
    predicted_values = regression_model.predict(X_pred)

    # Impute missing values in the original DataFrame
    df.loc[df[target_column].isna(), target_column] = predicted_values





In [ ]:
max_regression_ready_dict ={}
for month_name, month_df in max_temp_dict_by_month1.items():
    # Pivot the DataFrame
    pivoted_df = month_df.pivot_table(index='Station' , columns=['Date'], values='Max T')
    
    # Pivot the DataFrame
    #pivoted_df = month_df.pivot_table(index='Date' , columns=['Station'], values='Max T')
    
    
    # Replace 'NA' with actual NaN values
    pivoted_df.replace('NaN', np.nan, inplace=True)
    
    # Update the dictionary with the pivoted DataFrame
    max_regression_ready_dict[month_name] = pivoted_df
    


In [ ]:
max_regression_1_dict ={}
for month_name, month_df in max_regression_ready_dict.items():
    
    # Step 1: Find the column with the minimum number of missing values
    value_missing_onlyone_column = month_df.isna().sum().idxmin()
    
    #print(value_missing_onlyone_column)
    
    # Step 2: Calculate the average of the selected column
    average_value = month_df[value_missing_onlyone_column].mean()
    
    #print(average_value)

    # Step 3: Replace NaN values in the selected column with the calculated average
    month_df[value_missing_onlyone_column].fillna(average_value, inplace=True)
    max_regression_1_dict[month_name] = month_df
    #print(month_df)

In [ ]:

max_regression_12_dict ={}
for month_name, month_df in max_regression_1_dict.items():
    
    # Get a list of all column names except the target columns
    target_columns = month_df.columns[month_df.isna().any()].tolist()
    feature_columns = [col for col in month_df.columns if col not in target_columns]

    # Iterate over target columns and impute missing values using regression
    for target_col in target_columns:
        impute_missing_with_regression(month_df, target_col, feature_columns)
        
    max_regression_12_dict[month_name]=month_df




In [ ]:
max_regression_123_dict ={}
for month_name, month_df in max_regression_12_dict.items():
    df_melted_df = month_df.reset_index().melt(id_vars =['Station'], 
                         var_name ='Date', value_name = 'Max T')
    
    df_melted_df = df_melted_df[['Date', 'Station','Max T']]
    
    max_regression_123_dict[month_name] = df_melted_df

In [ ]:
# Concatenate all DataFrames into a single DataFrame
concatenated_df_filled_1990 = pd.concat(max_regression_123_dict.values(), ignore_index=True)

In [ ]:
station_filled_1990 = concatenated_df_filled_1990.set_index('Station')
station_sort_filled_1990 = station_filled_1990.sort_values(by=['Station', 'Date'], ascending=[True, True])





In [ ]:
print(initial_min_dict)

In [ ]:

station_sort_filled_1990.to_csv('G:/fresh_start/main_thesis/data/temp_data/station_sort_filled_1990.csv')
